# Settings

In [1]:
# import modules 
%matplotlib inline
from os.path import join as opj
import nipype.interfaces.spm as spm
from nipype.interfaces.spm import Realign
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import FreeSurferSource, SelectFiles, DataSink
from nipype.algorithms.rapidart import ArtifactDetect
from nipype.algorithms.misc import Gunzip
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.ants import Registration, ApplyTransforms
from nipype.interfaces.freesurfer import FSCommand
import nipype.interfaces.fsl as fsl         # fsl
import nipype.interfaces.freesurfer as fs   # freesurfer
import nipype.interfaces.io as nio          # i/o routines
import nipype.interfaces.utility as util    # utility
from nipype.interfaces.c3 import C3dAffineTool
import nipype.pipeline.engine as pe  # pypeline engine


/home/jan/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:278: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '
/home/jan/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
# SPM standalone
matlab_cmd = '/usr/local/spm12/run_spm12.sh /usr/local/MATLAB/MATLAB_Compiler_Runtime/v713 script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

In [ ]:
# FreeSurfer - Specify the location of the freesurfer folder
fs_dir = '/home/...../derivatives/derivatives/mindboggle/freesurfer_subjects' # placeholder!
FSCommand.set_default_subjects_dir(fs_dir)

# define experiment specific parameters

In [ ]:
experiment_dir = '/home/..../' # location of experiment folder
subject_list = [] # create the subject_list variable

output_dir = 'output_....' # name of output folder
working_dir = 'input_....' # name of working directory

#subject_list = ['sub-01', 'sub-02', 'sub-03', 'sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-09', 'sub-10']
subject_list = ['sub-01'] 
task_list = ['']

number_of_slices = 30                         # number of slices in volume
TR = 2.0                                      # time repetition of volume


# Import data using SelectFiles

In [ ]:
# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

# SelectFiles - to grab the data (alternativ to DataGrabber)
templates = {'func': '{subject_id}/func/sub*.nii.gz',
             'aparc_aseg': '/home/......../derivatives/derivatives/mindboggle/freesurfer_subjects/{subject_id}/mri/aparc+aseg.mgz',
             'brainmask': '/home/........./derivatives/derivatives/mindboggle/freesurfer_subjects/{subject_id}/mri/brainmask.mgz',
             'ribbon': '/home/............/derivatives/derivatives/mindboggle/freesurfer_subjects/{subject_id}/mri/ribbon.mgz'}

selectfiles = Node(SelectFiles(templates,
                               base_directory=experiment_dir),
                   name="selectfiles")

# Datasink - creates output folder for important outputs
datasink = Node(DataSink(base_directory=experiment_dir,
                         container=output_dir),
                name="datasink")

## Use the following DataSink output substitutions
substitutions = [('_subject_id_', 'sub-'),
                 ('_task_name_', '/task-'),
                 ('_fwhm_', 'fwhm-'),
                 ('_roi', ''),
                 ('_mcf', ''),
                 ('_st', ''),
                 ('_flirt', ''),
                 ('.nii_mean_reg', '_mean'),
                 ('.nii.par', '.par'),
                 ]
subjFolders = [('fwhm-%s/' % f, 'fwhm-%s_' % f) for f in fwhm]
substitutions.extend(subjFolders)
datasink.inputs.substitutions = substitutions

# create nodes

In [ ]:
smooth = pe.Node(interface=spm.Smooth(), name="smooth")
fwhmlist = [6]
smooth.iterables = ('fwhm', fwhmlist)

# Despike node - despike data
despike = MapNode(afni.Despike(outputtype='NIFTI'),
               name='despike', iterfield=['in_file'])

# Realign - correct for motion
realign = Node(spm.Realign(register_to_mean=True),
               name="realign")

# Artifact Detection - determine which of the images in the functional series
# are outliers. This is based on deviation in intensity or movement.
art = Node(ArtifactDetect(norm_threshold=1,
                          zintensity_threshold=3,
                          mask_type='file',
                          parameter_source='SPM',
                          use_differences=[True, False]),
           name="art")

# Gunzip - unzip the functinal image
gunzipfunc = Node(Gunzip(), name="gunzipfunc")

# MRIConvert - to unzip output files
mriconvert_complete_mask = Node(MRIConvert(out_type='nii'),
                     name='mriconvert_complete_mask')

# FreeSurferSource - Data grabber specific for FreeSurfer data
fssource = Node(nio.FreeSurferSource(subjects_dir=fs_dir),
                run_without_submitting=True,
                name='fssource')

# Binarize Cortex node - creates a binary map of cortical voxel
binarizeCortical = Node(fs.Binarize(out_type='nii.gz',
                                    match = [3,42],
                                    binary_file='binarized_cortical.nii.gz'),
                        name='binarizeCortical')

# Binarize Subcortex node - creates a binary map of subcortical voxel
binarizeSubcortical = Node(fs.Binarize(out_type='nii.gz',
                                       match = [10,49,  # Thalamus
                                                17,53,  # Hippocampus
                                                18,54,  # Amygdala
                                                26,58,  # AccumbensArea
                                                ],
                                    binary_file='binarized_subcortical.nii.gz'),
                       name='binarizeSubcortical')

# Brainmask node - combines cortical and subcortical binary maps
wholebrainmask = Node(fsl.BinaryMaths(operation='add',
                                 args='-bin'),
                 name='wholebrainmask')

applyVolTrans_complete_mask = Node(fs.ApplyVolTransform(reg_header=True,
                                  interp='nearest'),
                name='applyVolTrans_complete_mask')

# Dilate node - dilates the binary brain mask
dilate_complete_mask = Node(fs.Binarize(dilate=1,
                          min=0.5),
              name='dilate_complete_mask')

# Binarize Dialte node - binarizes dilated mask again after transformation
binarizeDilatedCompleteMask = Node(fs.Binarize(min=0.1, binary_file='dilated_complete_mask.nii'),
                           name='binarizeDilatedCompleteMask')

# Convert FreeSurfer's MGZ format into NIfTI format xxx
convert2nii = Node(fs.MRIConvert(out_type='nii'), name='convert2nii')

# Convert the BBRegister transformation to ANTS ITK format xxx
convert2itk = Node(C3dAffineTool(fsl2ras=True,
                                 itk_transform=True),
                   name='convert2itk')

# BBRegister node - register a volume to FreeSurfer's anatomy 
bbregister = Node(fs.BBRegister(init = 'fsl',
                                contrast_type = 't2',
                                out_fsl_file = True),
                  name='bbregister')

# Template to normalize to
template = 'Info.standard_image('MNI152_T1_1mm.nii.gz')'# welches Template?

# Registration (good) - computes registration between subject's structural and MNI template. xxx
antsreg = Node(Registration(args='--float',
                            collapse_output_transforms=True,
                            fixed_image=template,
                            initial_moving_transform_com=True,
                            num_threads=1,
                            output_inverse_warped_image=True,
                            output_warped_image=True,
                            sigma_units=['vox']*3,
                            transforms=['Rigid', 'Affine', 'SyN'],
                            terminal_output='file',
                            winsorize_lower_quantile=0.005,
                            winsorize_upper_quantile=0.995,
                            convergence_threshold=[1e-06],
                            convergence_window_size=[10],
                            metric=['MI', 'MI', 'CC'],
                            metric_weight=[1.0]*3,
                            number_of_iterations=[[1000, 500, 250, 100],
                                                  [1000, 500, 250, 100],
                                                  [100, 70, 50, 20]],
                            radius_or_number_of_bins=[32, 32, 4],
                            sampling_percentage=[0.25, 0.25, 1],
                            sampling_strategy=['Regular',
                                               'Regular',
                                               'None'],
                            shrink_factors=[[8, 4, 2, 1]]*3,
                            smoothing_sigmas=[[3, 2, 1, 0]]*3,
                            transform_parameters=[(0.1,),
                                                  (0.1,),
                                                  (0.1, 3.0, 0.0)],
                            use_histogram_matching=True,
                            write_composite_transform=True),
               name='antsreg')

# Concatenate BBRegister's and ANTS' transforms into a list
merge = Node(util.Merge(2), iterfield=['in2'], name='mergexfm')

# Transform the EPI images. First to anatomical and then to the target
warpall = MapNode(ApplyTransforms(args='--float',
                                  input_image_type=3,
                                  interpolation='Linear',
                                  invert_transform_flags=[False, False],
                                  num_threads=1,
                                  reference_image=template,
                                  terminal_output='file'),
                  name='warpall', iterfield=['input_image'])



# specify functions

In [ ]:
# Gets path to the aparc+aseg file
def get_aparc_aseg(files):
    for name in files:
        if 'aparc+aseg' in name:
            return name

# Gets path to the ribbon file (both hemispheres)
def get_ribbon(files):
    for name in files:
        if 'ribbon.mgz' in name:
            return name
        
# Gets path to the mask file        
def get_maskmask(file):
    return file[0]

# plot realign parameters
def plot_realign_parameters(subject_id, realigned_files):

    # Import necessary modules
    from os.path import join as opj 
    import numpy
    import matplotlib.pyplot as plt
    import glob

    experiment_dir = '/home/........' # path to your experiment directory
    working_dir = 'practice/workingdir_preproc' # path to the working directory
    
    # Load the estimated parameters
    list_realign_parameter_files=glob.glob(opj(experiment_dir, working_dir, "preproc_hoaf", "_subject_id_"+subject_id, "realign", "*.txt"))
    
        
    plot_realign_parameters=[]
    for realign_file in list_realign_parameter_files:
    
        movement=numpy.loadtxt(realign_file)
    
        plt.figure(figsize=(10,8))
        
        plt.axhline(0, color='red')
        
        # Create the plots with matplotlib
        plt.subplot(211)
        plt.title('translation')
        plt.ylabel('in mm')
        plt.plot(movement[:,0], label='x')
        plt.plot(movement[:,1], label='y')
        plt.plot(movement[:,2], label='z')
        
        plt.legend(loc='best', fancybox=True)
        
        plt.axhline(0, color='red', alpha=0.5)
        
        plt.subplot(212)
        plt.title('rotation')
        plt.ylabel('in degrees')
        plt.plot(movement[:,3], label='pitch')
        plt.plot(movement[:,4], label='roll')
        plt.plot(movement[:,5], label='yaw')
        
        plt.legend(loc='best', fancybox=True)
        
        plt.axhline(0, color='red', alpha=0.5)

        plot_realign_parameters=plt.tight_layout()
        
        plt.savefig(opj(experiment_dir, working_dir, "preproc_hoaf" "_subject_id_"+subject_id, "realign", realign_file+".png"), bbox_inches='tight')
        
    return plot_realign_parameters
    
# plot realing parameters - plot realignment parameters and save it later on
plot_realign_parameters = Node(Function(input_names=['subject_id', 'realigned_files'],
                               output_names=['plot_realign_parameters'],
                               function=plot_realign_parameters),
                      name='plot_realign_parameters')

# create and connect preprocessing workflow

In [ ]:
# Create a preprocessing workflow
preproc_hoaf = Workflow(name='preproc_hoaf')
preproc_hoaf.base_dir = opj(experiment_dir, working_dir)

# Connect all components of the preprocessing workflow - aparc / ribbon mask 
preproc_hoaf.connect([(gunzipfunc, realign, [('out_file', 'in_files')]), # send unzipped files to realignment node
                 (realign, art, [('realigned_files', 'realigned_files')]), # send realigned files to artifact detection node
                 (realign, art, [('mean_image', 'mask_file'),
                                 ('realignment_parameters',
                                  'realignment_parameters')]),
                 (realign, plot_realign_parameters, [('realigned_files', 'realigned_files')]), # send realign files / parameters to plot realignment parameters node 
                 (fssource,binarizeSubcortical,[(('aparc_aseg',         
                                                    get_aparc_aseg),
                                                   'in_file')]),   #??
                 (fssource,binarizeCortical,[(('ribbon',get_ribbon),
                                                'in_file')]),           # ?? get and binarize aparc_aseg and ribbon files from freesurfer
                 (binarizeCortical,wholebrainmask,[('binary_file','in_file')]),
                 (binarizeSubcortical,wholebrainmask,[('binary_file',
                                                   'operand_file')]), # combine aparc_aseg and ribbon files to get a whole brain mask    
                 (wholebrainmask, mriconvert_complete_mask, [('out_file', 'in_file')]), # convert whole brain mask to nifti
                 
                 (mriconvert_complete_mask, applyVolTrans_complete_mask,[('out_file','source_file')]), # transform complete mask to functional space
                 (realign, applyVolTrans_complete_mask, [('mean_image', 'target_file')]),
                 
                 (applyVolTrans_complete_mask, dilate_complete_mask, [('transformed_file', 'in_file')]), # dilate transformed whole brain mask
                 (dilate_complete_mask, binarizeDilatedCompleteMask, [('binary_file', 'in_file')]), # binarize dilated and transformed whole brain mask   
                 (fssource, convert2nii, [('brain', 'in_file')]), # xxx
                 (convert2nii, convert2itk, [('out_file', 'reference_file')]), # xxx
                 (bbregister, convert2itk, [('out_fsl_file',
                                              'transform_file')]), # xxx
                 (antsreg, merge, [('composite_transform', 'in1')]),
                 (convert2itk, merge, [('itk_transform', 'in2')]), 
                 (realign, convert2itk, [('mean_image', 'source_file')]), # x 
                 (realign, bbregister,[('mean_image','source_file')]), 
                 (merge, warpall, [('out', 'transforms')]),  
                 (despike, smooth, [('out_file', 'in_file')]),     
                 ]) 


In [ ]:
# visualize
# Create preproc output graph
preproc_hoaf.write_graph(graph2use='colored', format='png', simple_form=True)

# Visualize the graph
from IPython.display import Image
Image(filename='/......./graph.png', width=750)

# Connect workflow to SelectFiles and Datasink

In [ ]:
# Connect infosource, selectfiles and datasink to the preprocessing workflow
preproc_hoaf.connect([(infosource, selectfiles, [('subject_id', 'subject_id')]),
                  (infosource, fssource, [('subject_id',
                                          'subject_id')]),
                  (infosource, bbregister, [('subject_id',
                                          'subject_id')]), 
                  (infosource, plot_realign_parameters, [('subject_id', 'subject_id')]),		   
                  (selectfiles, gunzipfunc, [('func', 'in_file')]),
                  (selectfiles, gunzip_anat, [('anat', 'in_files')]),
                  (selectfiles, antsreg, [('anat', 'moving_image')]), # x    
                  (selectfiles, binarizeSubcortical, [('aparc_aseg', 'in_file')]),
                  (selectfiles, binarizeCortical, [('ribbon', 'in_file')]),       
                  (realign, datasink, [('mean_image',
                                        'realign.@mean'),
                                       ('realignment_parameters',
                                        'realign.@parameters')]),
                  (bbregister, datasink, [('out_reg_file',
                                        'bbregister.@out_reg_file'),
                                       ('out_fsl_file',
                                        'bbregister.@out_fsl_file'),
                                       ('registered_file',
                                        'bbregister.@registered_file')])
                  (convert2itk, datasink, [('itk_transform', 'convert2itk.@itk_transform')]), # x
                  (antsreg, datasink, [('warped_image',
                                        'antsreg.@warped_image'),
                                       ('inverse_warped_image',
                                        'antsreg.@inverse_warped_image'),
                                       ('composite_transform',
                                        'antsreg.@transform'),
                                       ('inverse_composite_transform',
                                        'antsreg.@inverse_transform')]),
                  (art, datasink,     [('outlier_files',
                                        'art.@outliers'),
                                       ('plot_files',
                                        'art.@plot')]),
                  (smooth, datasink, [('smoothed_files', 'preproc.@smooth')]),
                  (binarizeDilatedCompleteMask, datasink, [('binary_file',      
                                        'masks.@binarized_dilated_complete_mask')]),    
                  (warpall, datasink, [('output_image', 'warp_complete.@warpall')])   
                  ])

# Visualize the workflow

In [ ]:
# visualize
# Create preproc output graph
preproc.write_graph(graph2use='colored', format='png', simple_form=True)

# Visualize the graph
from IPython.display import Image
Image(filename='/......./graph.png', width=750)

In [ ]:
# run the workflow
preproc.run('MultiProc', plugin_args={'n_procs': 8})